In [ ]:
#模型下载，bge-rerank模型由北京智源研究院（BAAI）开发，适合中英文任务;还有COhere Rerank模型 
from modelscope import snapshot_download
model_dir = snapshot_download('BAAI/bge-reranker-large', cache_dir="D:\\AInewModels")



In [2]:
# 作用：
# 加载分词器（tokenizer）和模型（model）
# 设置模型为评估模式（model.eval()）
# 关键点：
# 该模型本质是一个序列分类模型，输出的是相关性分数（而非分类标签）
# 适用于 (query, document) 对的语义匹配
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model_name="D:/AInewModels/BAAI/bge-reranker-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()
# 作用：计算问题与答案的相关性分数 
# 流程：
    # 分词器将文本对转换为模型可处理的输入（包括填充和截断）
    # 模型输出原始 logits（未归一化的分数）
    # view(-1) 将结果展平为一维张量
# 输出解释：
    # 分数越高表示相关性越强（通常 >1 表示相关，<0 表示不相关）
pairs = [['what is panda?', 'The giant panda is a bear species endemic to China.']]
inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt')
scores = model(**inputs).logits.view(-1).float()
print(scores)  # 输出相关性分数

tensor([4.9538], grad_fn=<ViewBackward0>)


In [3]:
pairs = [
    ['what is panda?', 'The giant panda is a bear species endemic to China.'],  # 高相关
    ['what is panda?', 'Pandas are cute.'],                                     # 中等相关
    ['what is panda?', 'The Eiffel Tower is in Paris.']                        # 不相关
]
inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt')
scores = model(**inputs).logits.view(-1).float()
print(scores)  # 输出相关性分数

tensor([ 4.9538, -0.4951, -9.4803], grad_fn=<ViewBackward0>)
